#Paquetes necesarios

In [1]:
import cv2  
import math 
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="FALSE"
from ultralytics import YOLO

Desde cámara, detección con yolov8 y modelo nano

In [ ]:
# Carga del modelo
model = YOLO('yolov8n.pt')

# Nombre de las distintas clases
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]


# Captura desde lawebcam
vid = cv2.VideoCapture(0)
  
while(True):      
    # fotograma a fotograma
    ret, img = vid.read()
  
    # si hay imagen válida
    if ret:  
        # Perform inference on an image
        results = model(img, stream=True)
        
        # Para cada detección
        for r in results:
            boxes = r.boxes

            for box in boxes:
                # Contenedor
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values
                
                # Confianza
                confidence = math.ceil((box.conf[0]*100))/100
                print("Confidence --->",confidence)

                # Clase
                cls = int(box.cls[0])
                print("Class name -->", classNames[cls])

                # Convierte identificador numérico de clase a un color RGB
                escala = int((cls / len(classNames)) * 255 * 3)
                if escala >= 255*2:
                    R = 255
                    G = 255
                    B = escala - 255*2
                else:
                    if escala >= 255:
                        R = 255
                        G = escala - 255
                        B = 0
                    else:
                        R = escala
                        G = 0
                        B = 0

                # Dibuja el contenedor y clase
                cv2.rectangle(img, (x1, y1), (x2, y2), (R, G, B), 3)
                cv2.putText(img, classNames[cls] , [x1, y1], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, B), 2)

        # Muestra fotograma
        cv2.imshow('Vid', img)
    
    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break
  
# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()

Reconocimiento de caracteres tras instalar pytesseract y tesseract

In [ ]:
# Tesseract
import cv2
import pytesseract

# Previamente debes descargar los ejecutables
# Si la ruta de Tesseract no está en el PATH, ruta al ejecutable
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract'

# Lenguajes disponibles
print(pytesseract.get_languages(config=''))

#Cargo imagen y ocnvierto a RGB
img = cv2.imread('toy.tif') 
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#Aplica reconocedor a imagen cargada
print(pytesseract.image_to_string(img_rgb))

Reconocimiento decaracteres tras instalar easyocr

In [ ]:
import easyocr
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

#Carga del modelo de lengua
reader = easyocr.Reader(['es']) 

#Reconocimiento de una imagen
result = reader.readtext('toy.tif')
print(result)

#Con restricción de caracteres reconocibles
#result = reader.readtext('toy.tif', allowlist ='0123456789')

: 

In [15]:
# Carga del modelo
import cv2  
import math 
import numpy as np
import matplotlib.pyplot as plt
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="FALSE"
from ultralytics import YOLO
import easyocr


model = YOLO('./model/last.pt')

# Nombre de las distintas clases
classNames = ["License-Plate"]
reader = easyocr.Reader(['es']) 

img = cv2.imread("./testing/prueba4.jpeg")
# Captura desde lawebcam
results = model(img)

# Para cada detección de coche
for r in results:
    boxes = r.boxes

    for box in boxes:
        cls = int(box.cls[0])

        # Filtrar para mostrar solo las matrículas
        if classNames[cls] == "License-Plate":
            # Obtener datos de la detección
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # convertir a enteros
            plate= img[y1:y2, x1:x2]
            result = reader.readtext(plate)
            # Mostrar la caja delimitadora de la matrícula
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 3)
            cv2.putText(img, result[0][1], (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

# Guardar la imagen con las detecciones de matrículas
output_image_path = './Results/filtrado_con_matriculas.jpg'
cv2.imwrite(output_image_path, img)

print(f"Imagen con detecciones de matrículas guardada en: {output_image_path}")


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.

0: 416x640 1 License-Plate, 100.7ms
Speed: 0.0ms preprocess, 100.7ms inference, 0.0ms postprocess per image at shape (1, 3, 416, 640)


Imagen con detecciones de matrículas guardada en: ./Results/filtrado_con_matriculas.jpg
